In [1]:

import pandas as pd
import pywt
import numpy as np
import json

from tqdm import tqdm

# Load the datasets
games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/games.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/plays.csv')
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/players.csv')
player_play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/player_play.csv')


tracking_data_dict_play = {}

# for week in tqdm(range(6, 8)):
#     tracking_week_i = pd.read_csv(f'/kaggle/input/nfl-big-data-bowl-2025/tracking_week_{week}.csv')
  
tracking_data = pd.concat([
    pd.read_csv(f'/kaggle/input/nfl-big-data-bowl-2025/tracking_week_{week}.csv') for week in tqdm(range(1, 10))
#     pd.read_csv(f'/kaggle/input/nfl-big-data-bowl-2025/tracking_week_{week}.csv') for week in tqdm(range(1, 10))
])
        
    
dtype_dict = {'passResult':'str', 'foulName1':'str', 'foulName2': 'str'}

# for (gameId, playId), group_data in tracking_data.groupby(['gameId', 'playId']):
for (gameId, playId), group_data in tqdm(tracking_data.groupby(['gameId', 'playId']), total=tracking_data.groupby(['gameId', 'playId']).ngroups):

#     print('>',end="")

    if (gameId, playId) not in tracking_data_dict_play:
        tracking_data_dict_play[(gameId, playId)] = {
            'tracking_data': group_data,     
            'meta_data': {
                'gameId': gameId, 'playId': playId

            }
        }
    else:
        print('x',end="")

        tracking_data_dict_play[(gameId, playId)]['tracking_data'] = group_data
        tracking_data_dict_play[(gameId, playId)]['meta_data'] = {'gameId': gameId, 'playId': playId                                                                         
            }

                

100%|██████████| 16124/16124 [00:14<00:00, 1150.08it/s]


In [2]:
import torch
from torch.optim import LBFGS

import numpy as np

# def calculate_eccentricity(A, B, C):
def calculate_eccentricity(A, B, C, s, acc, dis, o, dir_, theta_) :
    # Step 1: Calculate eigenvalues for semi-major and semi-minor axis lengths
    eig_val1 = 0.5 * (A + C + np.sqrt((A - C)**2 + B**2))
    eig_val2 = 0.5 * (A + C - np.sqrt((A - C)**2 + B**2))
    a = np.sqrt(1 / np.abs(eig_val1))  # Major axis length
    b = np.sqrt(1 / np.abs(eig_val2))  # Minor axis length
#     a, b = sorted([a, b], reverse=True)  # Ensure a >= b
    
#     theta_deg = torch.degrees(theta_)
    theta_deg = torch.rad2deg(theta_)

 
    if torch.isclose((theta_deg % 180), torch.tensor(0.0, device=theta_deg.device), atol=5.0):
        orientation = "horizontal"
        eccentricity = a**2/b**2
#     elif np.isclose(theta_deg % 180, 90, atol=5):  # e.g., 85° to 95°
    elif torch.isclose((theta_deg % 180), torch.tensor(90.0, device=theta_deg.device), atol=5.0):
        orientation = "vertical"
        eccentricity = b**2/a**2
    else:
        orientation = "diagonal"  # if not close to 0° or 90°
        eccentricity = b**2/a**2
#         eccentricity = a**2/b**2
        
    return eccentricity#, is_horizontal, theta




# Objective function to minimize
def objective_fn(params, x, y):
    return torch.sum(conics(params, x, y)**2)






# def closure():
def closure(optimiser, sx ,sy, params ):
    optimizer.zero_grad()
    # Split params into sx and sy (assuming sx and sy are of the same size)
#     sx = params[:len(sx)]  # First part of params for sx
#     sy = params[len(sx):]  # Second part of params for sy

    # Now call the objective function with sx and sy
    loss = objective_fn(params, sx, sy)
    loss.backward()
    return loss



# Define conic equation
def conics(params, x, y):
    a, b, c, d, e, f = params
    return a * x**2 + b * x * y + c * y**2 + d * x + e * y + f

# # Objective function to minimize
# def objective_fn(params, x, y):
#     return np.sum(conics(params, x, y)**2)

def curvature_conic(A, B, C, D, E, x, y):
    # First derivative (dy/dx)
    numerator_first = -2*A*x - D
    denominator_first = B*x + 2*C*y + E
    dy_dx = numerator_first / denominator_first
    
    # Second derivative (d^2y/dx^2)
    # Derivatives of the numerator and denominator
    d_numerator_first_dx = -2*A
    d_denominator_first_dx = B
    
    # Using the quotient rule for the second derivative
    numerator_second = (d_numerator_first_dx * denominator_first) - (numerator_first * d_denominator_first_dx)
    denominator_second = denominator_first**2
    
    d2y_dx2 = numerator_second / denominator_second

    # Curvature calculation
    curvature = d2y_dx2 / (1 + dy_dx**2)**(3/2)
    
    return curvature

# def average_curvature(A, B, C, D, E, x_start, x_end, num_points=100):
#     x_values = np.linspace(x_start, x_end, num_points)
#     curvature_values = []
    
#     for x in x_values:
#         # To find corresponding y values, we would typically need to solve the conic equation.
#         # For simplicity, we can use a fixed value of y for average calculation, but this may not be accurate.
#         # Here, just for demonstration, we are considering y = 0, which might not always lie on the conic.
#         y = 0  
#         curv = curvature_conic(A, B, C, D, E, x, y)
#         curvature_values.append(curv)

#     # Calculate average curvature
#     average_curv = np.mean(curvature_values)
#     return average_curv

def average_curvature(A, B, C, D, E, x_start, x_end, num_points=100):
    # Create a tensor of x values, using torch.linspace
    x_values = torch.linspace(x_start, x_end, num_points, device='cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize the curvature_values tensor
    curvature_values = torch.zeros_like(x_values)
    
    # Compute the curvature for each x value
    for i, x in enumerate(x_values):
        y = 0  # Fixed y value for the calculation (this can be modified based on specific requirements)
        curvature_values[i] = curvature_conic(A, B, C, D, E, x, y)
    
    # Calculate the average curvature using torch.mean
    average_curv = torch.mean(curvature_values)
    
    return average_curv






# #hyperbolic only
# def calculate_eccentricity(A, B, C, s, acc, dis, o, dir_) :
def calculate_eccentricity_fx(A, B, C, s, acc, dis, o, dir_, theta_) :
    eig_val1 = 0.5 * (A + C + np.sqrt((A - C)**2 + B**2))
    eig_val2 = 0.5 * (A + C - np.sqrt((A - C)**2 + B**2))
    a = np.sqrt(1 / np.abs(eig_val1))
    b = np.sqrt(1 / np.abs(eig_val2))
#     a, b = sorted([a, b], key=abs)
#     a, b = sorted([a, b], key=abs, reverse=True)
#     a, b = sorted([a, b])
    a, b = sorted([a, b], reverse=True)
    
#         eccentricity = np.sqrt(1 - (b**2 / a**2))
#     eccentricity = np.sqrt(1 + (b**2 / a**2))
#     return np.sqrt(1 + (b**2 / a**2)) #
#     return (b**2 / a**2)
#     return np.log(b**2 / a**2)
#     return (a / b) #asect ratio
    return (a**2/b**2) # not ^
#     return 2*b**2/a  #latus rectum lr
#     return np.abs(theta_)
#     return B**2 - 4 * A * C #dis
#     return (b**2 / a**2) #asect ratio
#     return np.sqrt((b**2 / a**2))
#else return np.nan

In [3]:
# def calculate_conic_eccentricity(A, B, C):
#     discriminant = B**2 - 4*A*C
#     # D = B**2 - 4*A*C #discriminant

    
#     if discriminant < 0:
#         # Ellipse or Circle
#         eig_val1 = 0.5 * (A + C + np.sqrt((A - C)**2 + B**2))
#         eig_val2 = 0.5 * (A + C - np.sqrt((A - C)**2 + B**2))
#         a = np.sqrt(1 / np.abs(eig_val1))
#         b = np.sqrt(1 / np.abs(eig_val2))
#         eccentricity = np.sqrt(1 - (b**2 / a**2))
#         conic_type = "Ellipse" if a != b else "Circle"
#     elif discriminant == 0:
#         eccentricity = 1
#         conic_type = "Parabola"
#     else:
#         # Hyperbola
#         eig_val1 = 0.5 * (A + C + np.sqrt((A - C)**2 + B**2))
#         eig_val2 = 0.5 * (A + C - np.sqrt((A - C)**2 + B**2))
#         a = np.sqrt(1 / np.abs(eig_val1))
#         b = np.sqrt(1 / np.abs(eig_val2))
#         eccentricity = np.sqrt(1 + (b**2 / a**2))
#         conic_type = "Hyperbola"
    
#     return {"conic_type": conic_type, "eccentricity": eccentricity}


# def calculate_conic_eccentricity(A, B, C):
#     discriminant = B**2 - 4*A*C
#     # D = B**2 - 4*A*C #discriminant
#     eig_val1 = 0.5 * (A + C + np.sqrt((A - C)**2 + B**2))
#     eig_val2 = 0.5 * (A + C - np.sqrt((A - C)**2 + B**2))
#     a = np.sqrt(1 / np.abs(eig_val1))
#     b = np.sqrt(1 / np.abs(eig_val2))

#     if a > b:  # Ensure a is the semi-major axis
#         al = a
#     else:  # Handle edge cases where axes may be swapped
#         bl = b
    
#     if discriminant < 0:
#         # Ellipse or Circle

#         eccentricity = np.sqrt(1 - (bl**2 / al**2))
#         conic_type = "Ellipse" if a != b else "Circle"
#     elif discriminant == 0:
#         eccentricity = 1
#         conic_type = "Parabola"
#     else:
#         # Hyperbola

#         eccentricity = np.sqrt(1 + (bl**2 / al**2))
#         conic_type = "Hyperbola"
    
#     # return {"conic_type": conic_type, "eccentricity": eccentricity}
    # return eccentricity




def calculate_conic_eccentricity_extended(A, B, C, D, E, F):

    # discriminant
    discriminant = B**2 - 4*A*C
    # D = B**2 - 4*A*C #discriminant

    # determinant
    determinant = np.linalg.det([[A, B/2, D/2], 
                                 [B/2, C, E/2], 
                                 [D/2, E/2, F]])
    # determinant = np.linalg.det([[A, B/2, D/2], 
    #                              [B/2, C, E/2], 

    # # Check for degenerate conics
    # if determinant == 0:
    #     if discriminant < 0:
    #         conic_type = "Degenerate Ellipse (Point)"
    #     elif discriminant == 0:
    #         conic_type = "Degenerate Parabola (Parallel Lines)"
    #     elif discriminant > 0:
    #         conic_type = "Degenerate Hyperbola (Intersecting Lines)"
    #     # return {"conic_type": conic_type, "eccentricity": None}
    #     # return eccentricity
    #     return -1


                                                                             
    # axes lengths
    eig_val1 = 0.5 * (A + C + np.sqrt((A - C)**2 + B**2))
    eig_val2 = 0.5 * (A + C - np.sqrt((A - C)**2 + B**2))
    a = np.sqrt(1 / np.abs(eig_val1))
    b = np.sqrt(1 / np.abs(eig_val2))
    if a > b:  # Ensure a is the semi-major axis
        al = a
        bl = b
    else:  # Handle edge cases where axes may be swapped
        bl = a
        al = b



    
    if discriminant < 0:
        # Ellipse or Circle

        eccentricity = np.sqrt(1 - (bl**2 / al**2))
        conic_type = "Ellipse" if a != b else "Circle"
    elif discriminant == 0:
        eccentricity = 1
        conic_type = "Parabola"
    else:
        # Hyperbola

        eccentricity = np.sqrt(1 + (bl**2 / al**2))
        conic_type = "Hyperbola"
    
    # return {"conic_type": conic_type, "eccentricity": eccentricity}
    # return eccentricity
    # return (eccentricity, determinant, discriminant, conic_type)
    return eccentricity

In [4]:
import pandas as pd

games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/games.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/plays.csv')
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/players.csv')
player_play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/player_play.csv')

In [9]:
# params = p0.clone().detach().requires_grad_(True).to(device)
# params

In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.optimize import minimize


# # Objective function to minimize
# def objective_fn(params, x, y):
#     return np.sum(conics(params, x, y)**2)

# Objective function to minimize
def objective_fn(params, x, y):
    return torch.sum(conics(params, x, y)**2)


import torch
from torch.optim import LBFGS

# # Define the closure function for optimization
# def closure():
#     optimizer.zero_grad()  # Clear previous gradients
#     loss = objective_fn(params, sx, sy)  # Compute loss using objective_fn
#     loss.backward()  # Backpropagate the gradients
#     return loss


# def closure():
def closure(optimiser, sx ,sy, params ):
    optimizer.zero_grad()
    # Split params into sx and sy (assuming sx and sy are of the same size)
#     sx = params[:len(sx)]  # First part of params for sx
#     sy = params[len(sx):]  # Second part of params for sy

    # Now call the objective function with sx and sy
    loss = objective_fn(params, sx, sy)
    loss.backward()
    return loss



# Define conic equation
def conics(params, x, y):
    a, b, c, d, e, f = params
    return a * x**2 + b * x * y + c * y**2 + d * x + e * y + f

# # Objective function to minimize
# def objective_fn(params, x, y):
#     return np.sum(conics(params, x, y)**2)

def curvature_conic(A, B, C, D, E, x, y):
    # First derivative (dy/dx)
    numerator_first = -2*A*x - D
    denominator_first = B*x + 2*C*y + E
    dy_dx = numerator_first / denominator_first
    
    # Second derivative (d^2y/dx^2)
    # Derivatives of the numerator and denominator
    d_numerator_first_dx = -2*A
    d_denominator_first_dx = B
    
    # Using the quotient rule for the second derivative
    numerator_second = (d_numerator_first_dx * denominator_first) - (numerator_first * d_denominator_first_dx)
    denominator_second = denominator_first**2
    
    d2y_dx2 = numerator_second / denominator_second

    # Curvature calculation
    curvature = d2y_dx2 / (1 + dy_dx**2)**(3/2)
    
    return curvature

# def average_curvature(A, B, C, D, E, x_start, x_end, num_points=100):
#     x_values = np.linspace(x_start, x_end, num_points)
#     curvature_values = []
    
#     for x in x_values:
#         # To find corresponding y values, we would typically need to solve the conic equation.
#         # For simplicity, we can use a fixed value of y for average calculation, but this may not be accurate.
#         # Here, just for demonstration, we are considering y = 0, which might not always lie on the conic.
#         y = 0  
#         curv = curvature_conic(A, B, C, D, E, x, y)
#         curvature_values.append(curv)

#     # Calculate average curvature
#     average_curv = np.mean(curvature_values)
#     return average_curv

def average_curvature(A, B, C, D, E, x_start, x_end, num_points=100):
    # Create a tensor of x values, using torch.linspace
    x_values = torch.linspace(x_start, x_end, num_points, device='cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize the curvature_values tensor
    curvature_values = torch.zeros_like(x_values)
    
    # Compute the curvature for each x value
    for i, x in enumerate(x_values):
        y = 0  # Fixed y value for the calculation (this can be modified based on specific requirements)
        curvature_values[i] = curvature_conic(A, B, C, D, E, x, y)
    
    # Calculate the average curvature using torch.mean
    average_curv = torch.mean(curvature_values)
    
    return average_curv






# #hyperbolic only
# def calculate_eccentricity(A, B, C, s, acc, dis, o, dir_) :
def calculate_eccentricity_fx(A, B, C, s, acc, dis, o, dir_, theta_) :
    eig_val1 = 0.5 * (A + C + np.sqrt((A - C)**2 + B**2))
    eig_val2 = 0.5 * (A + C - np.sqrt((A - C)**2 + B**2))
    a = np.sqrt(1 / np.abs(eig_val1))
    b = np.sqrt(1 / np.abs(eig_val2))
#     a, b = sorted([a, b], key=abs)
#     a, b = sorted([a, b], key=abs, reverse=True)
#     a, b = sorted([a, b])
    a, b = sorted([a, b], reverse=True)
    
#         eccentricity = np.sqrt(1 - (b**2 / a**2))
#     eccentricity = np.sqrt(1 + (b**2 / a**2))
#     return np.sqrt(1 + (b**2 / a**2)) #
#     return (b**2 / a**2)
#     return np.log(b**2 / a**2)
#     return (a / b) #asect ratio
    return (a**2/b**2) # not ^
#     return 2*b**2/a  #latus rectum lr
#     return np.abs(theta_)
#     return B**2 - 4 * A * C #dis
#     return (b**2 / a**2) #asect ratio
#     return np.sqrt((b**2 / a**2))
#else return np.nan



# Initial guess for the conic parameters
# p0 = [1, 1, 1, 1, 1, 1]
p0 = torch.tensor([1, 1, 1, 1, 1, 1], dtype=torch.float32, requires_grad=True, device=device)


# Initialize the main dictionary to store no transforms
no_transforms = {}

# Summary stats for tracking processing
play_count_with_snap_event = 0
play_count_without_snap_event = 0
events_without_snap = {}

n = 500 #50 #500  # Subset for testing
# tracking_data_dict_play_subset = dict(list(tracking_data_dict_play.items())[:n])
tracking_data_dict_play_subset = dict(list(tracking_data_dict_play.items()))


# Device setup: Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


for (gameId, playId), group_data in tqdm(tracking_data_dict_play_subset.items()) :

    # Initialize gameId in no_transforms if not already present
    if gameId not in no_transforms:
        no_transforms[gameId] = {}

    # Initialize playId within the gameId if not already present
    if playId not in no_transforms[gameId]:
        no_transforms[gameId][playId] = {
            'offense': {},
            'defense': {},
        }

    
    offensiveTeamAbbr = plays[(plays['gameId'] == gameId) & (plays['playId'] == playId)].iloc[0]['possessionTeam']
    defensiveTeamAbbr = plays[(plays['gameId'] == gameId) & (plays['playId'] == playId)].iloc[0]['defensiveTeam']

    # Retrieve tracking data for the current game/play
    tracking_data = group_data['tracking_data']
    

    # Process tracking data for each player (nflId)
    for nflId, player_data in tracking_data.groupby('nflId'):
        if pd.isna(nflId):  # Skip if it's NaN (ball)
            continue

        ball_snap_idx = player_data[player_data['event'] == 'ball_snap'].index.min()
        snap_direct_idx = player_data[player_data['event'] == 'snap_direct'].index.min()

        snap_event_idx = min(ball_snap_idx, snap_direct_idx) if pd.notna(snap_direct_idx) and pd.notna(ball_snap_idx) else (ball_snap_idx if pd.notna(ball_snap_idx) else snap_direct_idx)

        
        
#         # Assuming player_data is sorted by time, so that the last row represents the latest event
#         last_event_idx = player_data.groupby('player_id')['event'].idxmax()  # Adjust to get the last event by time
#         last_event = player_data.loc[last_event_idx]



        if pd.notna(snap_event_idx):
            player_data_truncated = player_data.loc[:snap_event_idx]
            play_count_with_snap_event += 1
#         else : 
#             print("No")
#             player_data_truncated = None
            

            
            
            

#             x_ = player_data_truncated['x'].values
            x_ = torch.tensor(player_data_truncated['x'].values, device=device)
            y_ = torch.tensor(player_data_truncated['y'].values, device=device)
            s_ = torch.tensor(player_data_truncated['s'].values, device=device)
            a_ = torch.tensor(player_data_truncated['a'].values, device=device)
            dis_ = torch.tensor(player_data_truncated['dis'].values, device=device)
            dir_ = torch.tensor(player_data_truncated['dir'].values, device=device)
            
            o_ = torch.tensor(player_data_truncated['o'].values, device=device)

            #
# #             disp = np.sum(dis_) #$disall = np.sum(dis_)
#             disp = player_data_truncated['dis'].sum()  # Instead of np.sum
#             xl = x_[-1]
#             yl = y_[-1]
# #             x_mean = np.mean(x_)
# #             y_mean = np.mean(y_)
#             x_mean = player_data_truncated['x'].mean()
#             y_mean = player_data_truncated['y'].mean()
        
#             # Sum of 'dis' (already in CUDA if player_data_truncated['dis'] is transferred as tensor)
#             disp = dis_.sum()  # PyTorch automatically performs the sum on GPU

            #
#             # Last elements of x_ and y_ (directly available in GPU tensors)
#             xl = x_[-1]
#             yl = y_[-1]

#             # Mean calculations using PyTorch
#             x_mean = x_.mean()  # Mean calculated on GPU
#             y_mean = y_.mean()

            #
            #optimized version
            disp = dis_.sum()  # Sum of 'dis' directly on GPU
            xl = x_[-1]        # Last x-coordinate
            yl = y_[-1]        # Last y-coordinate
            x_mean = x_.mean() # Mean of x-coordinates on GPU
            y_mean = y_.mean() # Mean of y-coordinates on GPU



            #
#             dir_rad = np.radians(dir_)
#             sx = s_*np.sin(dir_rad) #np.mean(s_*np.sin()  
#             sy = s_*np.cos(dir_rad)
#             ax = a_*np.sin(dir_rad)
#             ay = a_*np.cos(dir_rad)
#             param_s = minimize(objective_fn, p0, args=(sx, sy), method='L-BFGS-B') #xs
#             param_a = minimize(objective_fn, p0, args=(sy, ay), method='L-BFGS-B')
#             sa,sb,sc,sd,se,sf = param_s.x #$params_s.
#             aa,ab,ac,ad,ae,af = param_a.x
#             theta_s = np.arctan2(sb,sa-sc)/2
#             theta_a = np.arctan2(ab,aa-ac)/2
            
#             # Convert dir_ to radians
#             dir_rad = torch.radians(dir_)

#             # Trigonometric operations (PyTorch versions)
#             sx = s_ * torch.sin(dir_rad)  # PyTorch handles this on GPU
#             sy = s_ * torch.cos(dir_rad)
#             ax = a_ * torch.sin(dir_rad)
#             ay = a_ * torch.cos(dir_rad)
#             param_s = minimize(objective_fn, p0, args=(sx.cpu().numpy(), sy.cpu().numpy()), method='L-BFGS-B')
#             param_a = minimize(objective_fn, p0, args=(sy.cpu().numpy(), ay.cpu().numpy()), method='L-BFGS-B')

#             # Extract parameters
#             sa, sb, sc, sd, se, sf = param_s.x
#             aa, ab, ac, ad, ae, af = param_a.x

#             # Trigonometric calculations
#             theta_s = torch.arctan2(torch.tensor(sb), torch.tensor(sa - sc)) / 2
#             theta_a = torch.arctan2(torch.tensor(ab), torch.tensor(aa - ac)) / 2
 


            dir_rad = torch.deg2rad(dir_) #not radians
            sx = s_ * torch.sin(dir_rad)  # PyTorch handles this on GPU
            sy = s_ * torch.cos(dir_rad)
            ax = a_ * torch.sin(dir_rad)
            ay = a_ * torch.cos(dir_rad)
            
            # Define initial parameters for optimization
#             params = torch.tensor(p0, requires_grad=True, device=device)
            params = p0.clone().detach().requires_grad_(True).to(device)

            
            # Create LBFGS optimizer
            optimizer = LBFGS([params])
            
#             # Step through the optimizer
#             optimizer.step(closure)
            
            # Use closure for (sx, sy)
            optimizer.step(lambda: closure(optimizer, sx, sy, params))
#             optimizer.step(lambda: closure(optimizer, sx, sy))

            # After optimization, the parameters are updated. You can now access the optimized values:
            optimized_params = params.detach()  # Detach the tensor from the computation graph
            
            # Extract the optimized parameters (sa, sb, sc, sd, se, sf) for further use
            sa, sb, sc, sd, se, sf = optimized_params.tolist()  # Convert tensor to list for easy usage

            # Perform any further calculations or use these parameters in your code
#             theta_s = torch.arctan2(sb, sa - sc) / 2
            theta_s = torch.arctan2(torch.tensor(sb, device=device), torch.tensor(sa - sc, device=device)) / 2

            
            
            
#             params = torch.tensor(p0, requires_grad=True, device=device)
            params = p0.clone().detach().requires_grad_(True).to(device)
            optimizer = LBFGS([params])
            optimizer.step(lambda: closure(optimizer, ax, ay, params))
            optimized_params = params.detach()
            aa, ab, ac, ad, ae, af = optimized_params.tolist()
#             theta_a = torch.arctan2(ab, aa - ac) / 2
            theta_a = torch.arctan2(torch.tensor(ab, device=device), torch.tensor(aa - ac, device=device)) / 2

            
            
            
#             params = torch.tensor(p0, requires_grad=True, device=device)
            params = p0.clone().detach().requires_grad_(True).to(device)
            optimizer = LBFGS([params])
            optimizer.step(lambda: closure(optimizer, x_, y_, params))   # xx or xa
            optimized_params = params.detach()
            a,b,c,d,e,f = optimized_params.tolist() # aa bb cc ..
#             theta_ = torch.arctan2(ab, aa - ac) / 2
            theta_ = torch.arctan2(torch.tensor(b, device=device), torch.tensor(a - c, device=device)) / 2
            cos_theta_, sin_theta_ = torch.cos(theta_), torch.sin(theta_)

            
            
            
#             params = minimize(objective_fn, p0, args=(x_, y_), method='L-BFGS-B')
#             a,b,c,d,e,f = params.x
        
#             theta_ = np.arctan2(b,a-c)/2
#             cos_theta_, sin_theta_ = np.cos(theta_), np.sin(theta_)

#             a_prime = a*cos_theta_**2 +  b*cos_theta_*sin_theta_ +  c*sin_theta_**2
#             b_prime = 2*(c-a)*cos_theta_*sin_theta_ + b*(cos_theta_**2 -sin_theta_**2)
#             c_prime = a*sin_theta_**2 - b*cos_theta_*sin_theta_ + c*cos_theta_**2

            
#             params = torch.tensor(p0, requires_grad=True, device=device)
#             optimizer = LBFGS([params])
#             optimizer.step(closure)
#             optimized_params = params.detach() 
#             sa, sb, sc, sd, se, sf = optimized_params.tolist()
#             theta_s = torch.arctan2(sb, sa - sc) / 2
#             theta_a = torch.arctan2(ab, aa - ac) / 2
            
            
#             eccentricity = calculate_eccentricity(a, b, c, np.mean(s_), np.mean(a_), np.mean(dis_), np.mean(o_), np.mean(dir_), theta_)
#             eccentricity = calculate_eccentricity(a, b, c, torch.mean(s_), torch.mean(a_), torch.mean(dis_), torch.mean(o_), torch.mean(dir_), theta_)

#             curvature = curvature_conic(a,b,c,d,e, np.mean(x_), np.mean(y_) )
#             curvature = curvature_conic(a, b, c, d, e, torch.mean(x_), torch.mean(y_))

#             avg_cuvx = average_curvature(a,b,c,d,e, x_[0], x_[-1] )
#             avg_cuvy = average_curvature(a,b,c,d,e, y_[0], y_[-1] )
#             discriminant = b**2 - 4 * a * c
            
#             eig_val1 = 0.5 * (a + c + np.sqrt((a - c)**2 + b**2))
#             eig_val2 = 0.5 * (a + c - np.sqrt((a - c)**2 + b**2))
#             al = np.sqrt(1 / np.abs(eig_val1))
#             bl = np.sqrt(1 / np.abs(eig_val2))
#             al, bl = sorted([al, bl], reverse=True)

    

#             arxy = calculate_eccentricity_fx(a, b, c, np.mean(s_), np.mean(a_), np.mean(dis_), np.mean(o_), np.mean(dir_), theta_) #aspect ratio
#             ars = calculate_eccentricity_fx(sa, sb, sc, np.mean(s_), np.mean(a_), np.mean(dis_), np.mean(o_), np.mean(dir_), theta_)
#             ara = calculate_eccentricity_fx(aa, ab, ac, np.mean(s_), np.mean(a_), np.mean(dis_), np.mean(o_), np.mean(dir_), theta_)
#             arxy = calculate_eccentricity_fx(a, b, c, torch.mean(s_), torch.mean(a_), torch.mean(dis_), torch.mean(o_), torch.mean(dir_), theta_) #aspect ratio
#             ars = calculate_eccentricity_fx(sa, sb, sc, torch.mean(s_), torch.mean(a_), torch.mean(dis_), torch.mean(o_), torch.mean(dir_), theta_)
#             ara = calculate_eccentricity_fx(aa, ab, ac, torch.mean(s_), torch.mean(a_), torch.mean(dis_), torch.mean(o_), torch.mean(dir_), theta_)


            
            if player_data['club'].iloc[0] == offensiveTeamAbbr:
#                 no_transforms[gameId][playId]['offense'][nflId] = {'eccentricity': eccentricity, 'disp':disp, 'theta': theta_, 'cuv':curvature, 'cuvx': avg_cuvx, 'cuvy': avg_cuvy, 'disc':discriminant, 'major':al, 'minor':bl, 'arxy':arxy, 'ars':ars, 'ara': ara, 'theta_s': theta_s, 'theta_a':theta_a, 'xl':xl, 'yl':yl, 'x_mean':x_mean, 'y_mean':y_mean, 'gameid': group_data['meta_data']['gameId'], 'playid': group_data['meta_data']['playId'], 'a'=a,'b'= b,'c'=c,'d'=d,'e'=e,'f'=f, 'sa'=sa, 'sb'=sb, 'sc'=sc, s'sd'=d, 'se'=se, 'sf'sf, 'aa'=aa, 'ab'=ab, 'ac'= ac, 'ad'=ad, 'ae'=ae, 'af'=af}
#                 no_transforms[gameId][playId]['offense'][nflId] = {'a'=a,'b'= b,'c'=c,'d'=d,'e'=e,'f'=f, 'sa'=sa, 'sb'=sb, 'sc'=sc, s'sd'=d, 'se'=se, 'sf'sf, 'aa'=aa, 'ab'=ab, 'ac'= ac, 'ad'=ad, 'ae'=ae, 'af'=af}  
                no_transforms[gameId][playId]['offense'][nflId] = {'disp':disp, 'a':a,'b': b,'c': c,'d': d,'e': e,'f': f, 'sa': sa, 'sb': sb, 'sc':sc, 'sd': sd, 'se':se, 'sf': sf, 'aa': aa, 'ab': ab, 'ac': ac, 'ad': ad, 'ae': ae, 'af': af, 'xl':xl, 'yl':yl, 'x_mean':x_mean, 'y_mean':y_mean, 'gameid': group_data['meta_data']['gameId'], 'playid': group_data['meta_data']['playId']} #, 'last_event'=last_event}  
                    
            elif player_data['club'].iloc[0] == defensiveTeamAbbr:
#                 no_transforms[gameId][playId]['defense'][nflId] = {'eccentricity': eccentricity, 'disp':disp, 'theta': theta_, 'cuv':curvature,  'cuvx': avg_cuvx, 'cuvy': avg_cuvy, 'disc':discriminant, 'major':al, 'minor':bl, 'arxy':arxy, 'ars':ars, 'ara': ara, 'theta_s': theta_s, 'theta_a':theta_a, 'xl':xl, 'yl':yl, 'x_mean':x_mean, 'y_mean':y_mean, 'gameid': group_data['meta_data']['gameId'], 'playid': group_data['meta_data']['playId']}
                no_transforms[gameId][playId]['defense'][nflId] = {'disp':disp, 'a':a,'b': b,'c': c,'d': d,'e': e,'f': f, 'sa': sa, 'sb': sb, 'sc':sc, 'sd': sd, 'se':se, 'sf': sf, 'aa': aa, 'ab': ab, 'ac': ac, 'ad': ad, 'ae': ae, 'af': af, 'xl':xl, 'yl':yl, 'x_mean':x_mean, 'y_mean':y_mean, 'gameid': group_data['meta_data']['gameId'], 'playid': group_data['meta_data']['playId']} #, 'last_event'=last_event}
                
        else:
            play_count_without_snap_event += 1
            events_without_snap[(gameId, playId)] = player_data['event'].unique()  # Store unique events

# Final summary
print(f"Total plays processed: {len(tracking_data_dict_play_subset)}")
print(f"Plays with 'ball_snap' or 'snap_direct' event: {play_count_with_snap_event}")
print(f"Plays without 'ball_snap' or 'snap_direct' event: {play_count_without_snap_event}")


#  12%|█▏        | 402/3347 [14:22<1:43:40,  2.11s/it]
#   0%|          | 13/3362 [01:03<4:29:05,  4.82s/it] #gpu








# import torch

# Save the no_transforms dictionary to a file
# torch.save(no_transforms, 'no_transforms.pth')
# torch.save(no_transforms, 'no_transforms[6-8].pth')
# torch.save(no_transforms, 'temp_full_.pth')
#  41%|████      | 1384/3362 [28:46<41:07,  1.25s/it] 
# 96.65 mb
# 250 mb 2w -> 1.3  gb for 9w  in gen

# # Load the dictionary from the file
# loaded_no_transforms = torch.load('no_transforms.pth')

# print(loaded_no_transforms)


In [11]:
theta_ = torch.arctan2(torch.tensor(b, device=device), torch.tensor(a - c, device=device)) / 2
theta_s = torch.arctan2(torch.tensor(sb, device=device), torch.tensor(sa - sc, device=device)) / 2
theta_a = torch.arctan2(torch.tensor(ab, device=device), torch.tensor(aa - ac, device=device)) / 2


In [12]:

s_ = torch.tensor([0.,0.,0.])
a_ = torch.tensor([0.,0.,0.])
o_ = torch.tensor([0.,0.,0.])
dir_ = torch.tensor([0.,0.,0.])
# theta_ = torch.tensor([0.,0.,0.])

In [ ]:
# import tqdm as twdm
from tqdm import tqdm

data_list = []

for gameId in tqdm(no_transforms, desc="Games", unit="game"):
#     print('YG')
# for gameId in no_transforms:
    for playId in no_transforms[gameId]: #2022101607 1127

#         print('YP')
#         for team_type in ['home', 'away', 'offense', 'defense']:
        for team_type in ['offense', 'defense']:
#             print('OD')
            for nflId, player_data in no_transforms[gameId][playId][team_type].items():
#                 print(nflId)
#                 if 'eccentricity' in player_data:

                a = player_data['a']
                b = player_data['b']
                c = player_data['c']
                d = player_data['d']
                e = player_data['e']
                f = player_data['f']

                theta_ = torch.arctan2(torch.tensor(player_data['b'], device=device), torch.tensor(player_data['a'] - player_data['c'], device=device)) / 2
                # print(theta_)
                theta_s = torch.arctan2(torch.tensor(player_data['sb'], device=device), torch.tensor(player_data['sa'] - player_data['sc'], device=device)) / 2
                theta_a = torch.arctan2(torch.tensor(player_data['ab'], device=device), torch.tensor(player_data['aa'] - player_data['ac'], device=device)) / 2

                eccentricity = calculate_eccentricity(player_data['a'], player_data['b'], player_data['c'], torch.mean(s_), torch.mean(a_), torch.mean(player_data['disp']), torch.mean(o_), torch.mean(dir_), theta_)
                eccentricity_s = calculate_eccentricity(player_data['sa'], player_data['sb'], player_data['sc'], torch.mean(s_), torch.mean(a_), torch.mean(player_data['disp']), torch.mean(o_), torch.mean(dir_), theta_)
                eccentricity_a = calculate_eccentricity(player_data['aa'], player_data['ab'], player_data['ac'], torch.mean(s_), torch.mean(a_), torch.mean(player_data['disp']), torch.mean(o_), torch.mean(dir_), theta_)
                # theta_ = torch.arctan2(torch.tensor(player_data['b'], device=device), torch.tensor(player_data['a'] - player_data['c'], device=device)) / 2
                # print(theta_)
                # theta_s = torch.arctan2(torch.tensor(player_data['sb'], device=device), torch.tensor(player_data['sa'] - player_data['sc'], device=device)) / 2
                # theta_a = torch.arctan2(torch.tensor(player_data['ab'], device=device), torch.tensor(player_data['aa'] - player_data['ac'], device=device)) / 2


                eccentricity_extended = calculate_conic_eccentricity_extended(player_data['a'], player_data['b'], player_data['c'], player_data['d'], player_data['e'], player_data['f'])#calculate_eccentricity_extended
                eccentricity_s_extended = calculate_conic_eccentricity_extended(player_data['sa'], player_data['sb'], player_data['sc'], player_data['sd'], player_data['se'], player_data['sf'])#calculate_eccentricity_extended
                eccentricity_a_extended = calculate_conic_eccentricity_extended(player_data['aa'], player_data['ab'], player_data['ac'], player_data['ad'], player_data['af'], player_data['af']) #calculate_eccentricity_extended

                ### XYZ CUVS TODO CL

                
                disp = player_data['disp']


                arxy = calculate_eccentricity_fx(player_data['a'], player_data['b'], player_data['c'], torch.mean(s_), torch.mean(a_), torch.mean(disp), torch.mean(o_), torch.mean(dir_), theta_)
                ars = calculate_eccentricity_fx(player_data['sa'], player_data['sb'], player_data['sc'], torch.mean(s_), torch.mean(a_), torch.mean(disp), torch.mean(o_), torch.mean(dir_), theta_s)
                ara = calculate_eccentricity_fx(player_data['aa'], player_data['ab'], player_data['ac'], torch.mean(s_), torch.mean(a_), torch.mean(disp), torch.mean(o_), torch.mean(dir_), theta_a)


                
                try:
                    route_ran = player_play.loc[(player_play['gameId'] == gameId) &
                                                (player_play['playId'] == playId) &
                                                (player_play['nflId'] == nflId), 'routeRan'].values[0]

                    hadRushAttempt_ = player_play.loc[(player_play['gameId'] == gameId) &
                                                (player_play['playId'] == playId) &
                                                (player_play['nflId'] == nflId), 'hadRushAttempt'].values[0]
                    rushingYards_ = player_play.loc[(player_play['gameId'] == gameId) &
                                                (player_play['playId'] == playId) &
                                                (player_play['nflId'] == nflId), 'rushingYards'].values[0]

                    pos = players.loc[players['nflId'] == nflId, 'position'].values[0] #, # cant speak why becomes lal on coment
                    
                    # eccentricity = calculate_eccentricity(player_data['a'], player_data['b'], player_data['c'], torch.mean(s_), torch.mean(a_), torch.mean(player_data['disp']), torch.mean(o_), torch.mean(dir_), theta_)
                    # eccentricity_s = calculate_eccentricity(player_data['sa'], player_data['sb'], player_data['sc'], torch.mean(s_), torch.mean(a_), torch.mean(player_data['disp']), torch.mean(o_), torch.mean(dir_), theta_)
                    # eccentricity_a = calculate_eccentricity(player_data['aa'], player_data['ab'], player_data['ac'], torch.mean(s_), torch.mean(a_), torch.mean(player_data['disp']), torch.mean(o_), torch.mean(dir_), theta_)
                    # theta_ = torch.arctan2(torch.tensor(player_data['b'], device=device), torch.tensor(player_data['a'] - player_data['c'], device=device)) / 2
                    # theta_s = torch.arctan2(torch.tensor(player_data['sb'], device=device), torch.tensor(player_data['sa'] - player_data['sc'], device=device)) / 2
                    # theta_a = torch.arctan2(torch.tensor(player_data['ab'], device=device), torch.tensor(player_data['aa'] - player_data['ac'], device=device)) / 2
       #week quarter
                    
                    w = games.loc[games['gameId'] == gameId, 'week'].values[0]#,
                    
                    q = plays.loc[(plays['gameId'] == gameId) &
                                                (plays['playId'] == playId), 'quarter'].values[0]#,
                    
                    d = plays.loc[(plays['gameId'] == gameId) &
                                                (plays['playId'] == playId), 'down'].values[0]

#                     eccentricity
                    # arxy = calculate_eccentricity_fx(player_data['a'], player_data['b'], player_data['c'], torch.mean(s_), torch.mean(a_), torch.mean(disp), torch.mean(o_), torch.mean(dir_), theta_)
                    # ars = calculate_eccentricity_fx(player_data['sa'], player_data['sb'], player_data['sc'], torch.mean(s_), torch.mean(a_), torch.mean(disp), torch.mean(o_), torch.mean(dir_), theta_s)
                    # ara = calculate_eccentricity_fx(player_data['aa'], player_data['ab'], player_data['ac'], torch.mean(s_), torch.mean(a_), torch.mean(disp), torch.mean(o_), torch.mean(dir_), theta_a)
            
                except IndexError:
                    print(gameId, playId)
                    route_ran = None  # In case routeRan is missing

                    hadRushAttempt_ = None
                    rushingYards_ = None

                    pos = None
                    
                    w = None
                    q = None
                    d = None
                    # arxy = None
                    # ars = None
                    # ara = None


                    # eccentricity = None
                    # eccentricity_s = None
                    # eccentricity_a = None
                    # theta_ = None
                    # theta_s = None
                    # theta_a = None


                # eccentricity = calculate_eccentricity(player_data['a'], player_data['b'], player_data['c'], torch.mean(s_), torch.mean(a_), torch.mean(player_data['disp']), torch.mean(o_), torch.mean(dir_), theta_)
                # eccentricity_s = calculate_eccentricity(player_data['sa'], player_data['sb'], player_data['sc'], torch.mean(s_), torch.mean(a_), torch.mean(player_data['disp']), torch.mean(o_), torch.mean(dir_), theta_)
                # eccentricity_a = calculate_eccentricity(player_data['aa'], player_data['ab'], player_data['ac'], torch.mean(s_), torch.mean(a_), torch.mean(player_data['disp']), torch.mean(o_), torch.mean(dir_), theta_)
                # theta_ = torch.arctan2(torch.tensor(player_data['b'], device=device), torch.tensor(player_data['a'] - player_data['c'], device=device)) / 2
                # theta_s = torch.arctan2(torch.tensor(player_data['sb'], device=device), torch.tensor(player_data['sa'] - player_data['sc'], device=device)) / 2
                # theta_a = torch.arctan2(torch.tensor(player_data['ab'], device=device), torch.tensor(player_data['aa'] - player_data['ac'], device=device)) / 2

        
                data_list.append({
                    'gameId': gameId,
                    'playId': playId,
                    'nflId': nflId,

#                     'eccentricity': player_data['eccentricity'],
                    'routeRan': route_ran,
                    'rushingYards': rushingYards_,

                    'hadRushAttempt': hadRushAttempt_,
                    
#                         'dis': player_data['dis'].mean()
                    'disp': player_data['disp'], #,
                    'week': w,
                    'quarter': q,
                    'down' : d,
                    'arxy': arxy,
                    'ars': ars,
                    'ara': ara,
                    'pos' : pos,

                    'eccentricity' : eccentricity,
                    'eccentricity_s' : eccentricity_s,
                    'eccentricity_a' : eccentricity_a,
                    'theta_' : theta_,
                    'theta_s' : theta_s,
                    'theta_a' : theta_a,


                    'eccentricity_extended' : eccentricity_extended,
                    'eccentricity_extended_s' : eccentricity_s_extended,
                    'eccentricity_extended_a' : eccentricity_a_extended,


                    'a' : a,
                    'b' : b,
                    'c' : c,
                    'd' : d,
                    'e' : e,
                    'f' : f
                    # CAN CUVS
                    ## add names
                    
#                         'quarter' : 
 #                     #'theta': player_data['theta']
#                     'theta': np.degrees(player_data['theta']),
#                     'theta_abs': np.abs(np.degrees(player_data['theta'])),
#                     'curvature' : player_data['cuv'],
#                     'curvaturex' : player_data['cuvx'],
#                     'curvaturey' : player_data['cuvy'],
#                     'curvaturexabs' : np.abs(player_data['cuvx']),
#                     'discriminant' : player_data['disc'],


#                     #'rushingYards' : rushingYards_, 'disp':disp, 'week': w, 'quarter':q, 'theta': theta_, 'cuv':curvature, 'cuvx': avg_cuvx, 'cuvy': avg_cuvy, 'disc':discriminant, 
#                     'major':player_data['major'], 'minor':player_data['minor'], 'arxy':player_data['arxy'], 'ars':player_data['ars'], 'ara': player_data['ara'], 'theta_s': np.degrees(player_data['theta_s']), 'theta_a':np.degrees(player_data['theta_a']), 'xl':player_data['xl'], 'yl':player_data['yl'], 'x_mean':player_data['x_mean'], 'y_mean':player_data['y_mean']

#                         'dir'
                })
                #wkk2 order of [''] pairing, : annotation  'major' as key rather no quotes



# torch.save(no_transforms, 'no_transforms.pth')



# Create a DataFrame from the list
eccentricity_df = pd.DataFrame(data_list)
torch.save(eccentricity_df, 'eccentricity_df.pth')

eccentricity_df.head()

In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the eccentricity_df if not already loaded
# eccentricity_df = torch.load('/kaggle/working/eccentricity_df.pth')

# Ensure eccentricity_df is a DataFrame (if it isn't already)
if not isinstance(eccentricity_df, pd.DataFrame):
    eccentricity_df = pd.DataFrame(eccentricity_df)

# Assuming player_play is another DataFrame already loaded, with relevant data
# Use tqdm to add a progress bar
for idx, row in tqdm(eccentricity_df.iterrows(), total=eccentricity_df.shape[0], desc="Updating passResult"):
    gameId = row['gameId']
    playId = row['playId']
    nflId = row['nflId']
    
    try:
        # Extract the 'passResult' for the given gameId, playId, and nflId
        pass_result = player_play.loc[
            (player_play['gameId'] == gameId) &
            (player_play['playId'] == playId) &
            (player_play['nflId'] == nflId), 'rushingYards'
        ].values[0]  # Assuming 'passResult' exists in player_play

        pass_result = player_play.loc[
            (player_play['gameId'] == gameId) &
            (player_play['playId'] == playId) &
            (player_play['nflId'] == nflId), 'passingYards'
        ].values[0]  # Assuming 'passResult' exists in player_play

        
        # Add 'passResult' to the eccentricity_df
        eccentricity_df.at[idx, 'rushingYards'] = pass_result
        eccentricity_df.at[idx, 'passingYards'] = pass_result
    except IndexError:
        # Handle case where no matching 'passResult' is found
        eccentricity_df.at[idx, 'rushingYards'] = None
        eccentricity_df.at[idx, 'passingYards'] = None
        
# Optionally, save the updated dataframe
# torch.save(eccentricity_df, '/kaggle/working/eccentricity_df_with_passResult.pth')


In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the eccentricity_df if not already loaded
# eccentricity_df = torch.load('/kaggle/working/eccentricity_df.pth')

# Ensure eccentricity_df is a DataFrame (if it isn't already)
if not isinstance(eccentricity_df2, pd.DataFrame):
    eccentricity_df2 = pd.DataFrame(eccentricity_df2)

# Assuming player_play is another DataFrame already loaded, with relevant data
# Use tqdm to add a progress bar
for idx, row in tqdm(eccentricity_df2.iterrows(), total=eccentricity_df2.shape[0], desc="Updating passResult"):
    gameId = row['gameId']
    playId = row['playId']
    nflId = row['nflId']
    
    try:
        # Extract the 'passResult' for the given gameId, playId, and nflId
        wasRunningRoute_result = player_play.loc[
            (player_play['gameId'] == gameId) &
            (player_play['playId'] == playId) &
            (player_play['nflId'] == nflId), 'wasRunningRoute'
        ].values[0]  # Assuming 'passResult' exists in player_play
        
        # Add 'passResult' to the eccentricity_df
        eccentricity_df2.at[idx, 'wasRunningRoute'] = wasRunningRoute_result
    except IndexError:
        # Handle case where no matching 'passResult' is found
        eccentricity_df2.at[idx, 'wasRunningRoute'] = None

        
# Optionally, save the updated dataframe
# torch.save(eccentricity_df, '/kaggle/working/eccentricity_df_with_passResult.pth')
torch.save(eccentricity_df2, '/kaggle/working/eccentricity_df3.pth')


In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the eccentricity_df if not already loaded
# eccentricity_df = torch.load('/kaggle/working/eccentricity_df.pth')

# Ensure eccentricity_df is a DataFrame (if it isn't already)
if not isinstance(eccentricity_df, pd.DataFrame):
    eccentricity_df = pd.DataFrame(eccentricity_df)

# Assuming player_play is another DataFrame already loaded, with relevant data
# Use tqdm to add a progress bar
for idx, row in tqdm(eccentricity_df.iterrows(), total=eccentricity_df.shape[0], desc="Updating passResult"):
    gameId = row['gameId']
    playId = row['playId']
    nflId = row['nflId']
    
    try:
        # Extract the 'passResult' for the given gameId, playId, and nflId
        wasRunningRoute_result = player_play.loc[
            (player_play['gameId'] == gameId) &
            (player_play['playId'] == playId) &
            (player_play['nflId'] == nflId), 'wasRunningRoute'
        ].values[0]  # Assuming 'passResult' exists in player_play
        
        # Add 'passResult' to the eccentricity_df
        eccentricity_df.at[idx, 'wasRunningRoute'] = wasRunningRoute_result
    except IndexError:
        # Handle case where no matching 'passResult' is found
        eccentricity_df.at[idx, 'wasRunningRoute'] = None

        
# Optionally, save the updated dataframe
# torch.save(eccentricity_df, '/kaggle/working/eccentricity_df_with_passResult.pth')
torch.save(eccentricity_df, '/kaggle/working/eccentricity_df3.pth')
